# Libraries

In [22]:
# !pip install geopy
import requests
from geopy.geocoders import Nominatim
import pandas as pd
import requests
import time

# Reading from the CSVs

In [23]:
# Create a list of all the individual DataFrames
dfs = [pd.read_csv(f"../../Data/Police_Data/Crime_Data/crime_2021/2021-{month:02}-metropolitan-street.csv") for month in range(1, 13)]

In [24]:
count = 0 
for df in dfs:
    count += df.shape[0]
    print(df.shape[0])

print("Total Instances:",count)

84848
79974
91666
87199
92225
95527
94605
89240
90881
93992
90141
82578
Total Instances: 1072876


## Dropping Duplicates

In [26]:
# print number of duplicates in the dataset
countt = 0
for df in dfs:
    print(df.duplicated().sum())
    countt += df.duplicated().sum()
print("Total Duplicates:",countt)

18679
15134
16330
13880
13377
14614
13532
11756
10888
10324
8715
8037
Total Duplicates: 155266


In [27]:
# Drop duplicates
for df in dfs:
    df.drop_duplicates(inplace=True)

In [31]:
for df in dfs:
    print(df.duplicated().sum())

0
0
0
0
0
0
0
0
0
0
0
0


## Dropping unnecessary columns

In [32]:
for df in dfs:
    df.drop(columns=['Crime ID','Reported by','Falls within','LSOA code','LSOA name','Crime type','Last outcome category','Context','Location'], inplace=True)

In [33]:
dfs[0].head()   

,Month,Longitude,Latitude
0,2021-01,0.880386,51.219190
1,2021-01,0.145888,51.593835
2,2021-01,0.134947,51.588063
5,2021-01,0.140192,51.582311
7,2021-01,0.141143,51.590873


## Dropping missing values

In [36]:
# null values in the dataset
for df in dfs:
    print(df.isna().sum())

Month          0
Longitude    317
Latitude     317
dtype: int64
Month          0
Longitude    318
Latitude     318
dtype: int64
Month          0
Longitude    829
Latitude     829
dtype: int64
Month          0
Longitude    838
Latitude     838
dtype: int64
Month          0
Longitude    960
Latitude     960
dtype: int64
Month           0
Longitude    1075
Latitude     1075
dtype: int64
Month          0
Longitude    954
Latitude     954
dtype: int64
Month           0
Longitude    1605
Latitude     1605
dtype: int64
Month           0
Longitude    1954
Latitude     1954
dtype: int64
Month           0
Longitude    2096
Latitude     2096
dtype: int64
Month           0
Longitude    2158
Latitude     2158
dtype: int64
Month           0
Longitude    1986
Latitude     1986
dtype: int64


In [37]:
# Drop null values
for df in dfs:
    df.dropna(inplace=True)

# Converting the coordinates to location

In [40]:
def request_with_rate_limit(url, params=None, username=None, password=None):
    auth = (username, password) if username and password else None
    response = requests.get(url, params=params, auth=auth)
    if response.status_code == 429:
        seconds_to_wait = int(response.headers["Retry-After"])
        time.sleep(seconds_to_wait)
        return request_with_rate_limit(url, params=params, auth=auth)
    return response

counter = 0
def get_location(row):
    global counter
    counter += 1
    print(counter)
    lat = row["Latitude"]
    lng = row["Longitude"]

    try:
        nominatim_url = "https://nominatim.openstreetmap.org/reverse"
        nominatim_params = {"lat": lat, "lon": lng, "format": "json"}
        response = request_with_rate_limit(nominatim_url, params=nominatim_params)
        response.raise_for_status()
        location = response.json()
        if 'postcode' in location['address']:
            postcode = location['address']['postcode']
        else:
            return "No postal code found"
        if postcode:
            url = f"https://api.postcodes.io/postcodes/{postcode}"
            try:
                response = requests.get(url, timeout=10)
                response.raise_for_status()
                data = response.json()
                if data.get('result', {}).get('admin_district'):
                    borough = data['result']['admin_district']
                else:
                    borough = None
                print(borough)
                return borough
            except (requests.exceptions.RequestException, KeyError):
                pass
        return None
    except (requests.exceptions.RequestException, KeyError):
        return None


In [ ]:
dfs[0]["Borough"] = dfs[0].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[0].to_csv('clean_met_2021_01.csv', index=False)

1
Ashford
2
Barking and Dagenham
3
Barking and Dagenham
4
Barking and Dagenham
5
Barking and Dagenham
6
Barking and Dagenham
7
Barking and Dagenham
8
Barking and Dagenham
9
Barking and Dagenham
10
Barking and Dagenham
11
Barking and Dagenham
12
Barking and Dagenham
13
Barking and Dagenham
14
Barking and Dagenham
15
Barking and Dagenham
16
Barking and Dagenham
17
Barking and Dagenham
18
Barking and Dagenham
19
Barking and Dagenham
20
Barking and Dagenham
21
Barking and Dagenham
22
Barking and Dagenham
23
Barking and Dagenham
24
Barking and Dagenham
25
Barking and Dagenham
26
Barking and Dagenham
27
Barking and Dagenham
28
Barking and Dagenham
29
Barking and Dagenham
30
Barking and Dagenham
31
Barking and Dagenham
32
Barking and Dagenham
33
Barking and Dagenham
34
Barking and Dagenham
35
Barking and Dagenham
36
Barking and Dagenham
37
Barking and Dagenham
38
Barking and Dagenham
39
Barking and Dagenham
40
Barking and Dagenham
41
Redbridge
42
Barking and Dagenham
43
Barking and Dagenham
4

Barking and Dagenham
345
Barking and Dagenham
346
Barking and Dagenham
347
Barking and Dagenham
348
Barking and Dagenham
349
Barking and Dagenham
350
Barking and Dagenham
351
Barking and Dagenham
352
Barking and Dagenham
353
Barking and Dagenham
354
Barking and Dagenham
355
Barking and Dagenham
356
Barking and Dagenham
357
Barking and Dagenham
358
Barking and Dagenham
359
Barking and Dagenham
360
Barking and Dagenham
361
Barking and Dagenham
362
Barking and Dagenham
363
364
Barking and Dagenham
365
366
Barking and Dagenham
367
Barking and Dagenham
368
Barking and Dagenham
369
370
Barking and Dagenham
371
372
Barking and Dagenham
373
Barking and Dagenham
374
Barking and Dagenham
375
Barking and Dagenham
376
377
Barking and Dagenham
378
Barking and Dagenham
379
Barking and Dagenham
380
Barking and Dagenham
381
Barking and Dagenham
382
Barking and Dagenham
383
Barking and Dagenham
384
Barking and Dagenham
385
Barking and Dagenham
386
Barking and Dagenham
387
Barking and Dagenham
388
Barki

Barking and Dagenham
681
Barking and Dagenham
682
Barking and Dagenham
683
Barking and Dagenham
684
Barking and Dagenham
685
Barking and Dagenham
686
Barking and Dagenham
687
Barking and Dagenham
688
Barking and Dagenham
689
Barking and Dagenham
690
Barking and Dagenham
691
Barking and Dagenham
692
Barking and Dagenham
693
Barking and Dagenham
694
Barking and Dagenham
695
Barking and Dagenham
696
Barking and Dagenham
697
Barking and Dagenham
698
Barking and Dagenham
699
Barking and Dagenham
700
Barking and Dagenham
701
Barking and Dagenham
702
Barking and Dagenham
703
Barking and Dagenham
704
Barking and Dagenham
705
Barking and Dagenham
706
Barking and Dagenham
707
Barking and Dagenham
708
Barking and Dagenham
709
Barking and Dagenham
710
Barking and Dagenham
711
Barking and Dagenham
712
Barking and Dagenham
713
Barking and Dagenham
714
Barking and Dagenham
715
Barking and Dagenham
716
Barking and Dagenham
717
Barking and Dagenham
718
Barking and Dagenham
719
Barking and Dagenham
720


Barking and Dagenham
1014
Barking and Dagenham
1015
Barking and Dagenham
1016
Barking and Dagenham
1017
Barking and Dagenham
1018
Barking and Dagenham
1019
Barking and Dagenham
1020
Barking and Dagenham
1021
Barking and Dagenham
1022
Barking and Dagenham
1023
Barking and Dagenham
1024
Barking and Dagenham
1025
Barking and Dagenham
1026
Barking and Dagenham
1027
Barking and Dagenham
1028
Barking and Dagenham
1029
Barking and Dagenham
1030
Barking and Dagenham
1031
Barking and Dagenham
1032
Barking and Dagenham
1033
Barking and Dagenham
1034
Barking and Dagenham
1035
Barking and Dagenham
1036
Barking and Dagenham
1037
Barking and Dagenham
1038
Barking and Dagenham
1039
Barking and Dagenham
1040
Barking and Dagenham
1041
Barking and Dagenham
1042
Barking and Dagenham
1043
Barking and Dagenham
1044
Barking and Dagenham
1045
Barking and Dagenham
1046
Barking and Dagenham
1047
Barking and Dagenham
1048
Barking and Dagenham
1049
Barking and Dagenham
1050
Barking and Dagenham
1051
Barking and 

Barking and Dagenham
1330
Barking and Dagenham
1331
Barking and Dagenham
1332
Barking and Dagenham
1333
Barking and Dagenham
1334
Barking and Dagenham
1335
Barking and Dagenham
1336
Barking and Dagenham
1337
Barking and Dagenham
1338
Barking and Dagenham
1339
Barking and Dagenham
1340
Barking and Dagenham
1341
Barking and Dagenham
1342
Barking and Dagenham
1343
Barking and Dagenham
1344
Barking and Dagenham
1345
Barking and Dagenham
1346
Barking and Dagenham
1347
Barking and Dagenham
1348
Barking and Dagenham
1349
Barking and Dagenham
1350
Barking and Dagenham
1351
Barking and Dagenham
1352
Barking and Dagenham
1353
Barking and Dagenham
1354
Barking and Dagenham
1355
Barking and Dagenham
1356
Barking and Dagenham
1357
Barking and Dagenham
1358
Barking and Dagenham
1359
Barking and Dagenham
1360
Barking and Dagenham
1361
Barking and Dagenham
1362
Barking and Dagenham
1363
Barking and Dagenham
1364
Barking and Dagenham
1365
Barking and Dagenham
1366
Barking and Dagenham
1367
Barking and 

...

Southwark
52946
Southwark
52947
Southwark
52948
Southwark
52949
Southwark
52950
Southwark
52951
Southwark
52952
Southwark
52953
Southwark
52954
Southwark
52955
Southwark
52956
Southwark
52957
Southwark
52958
Southwark
52959
Southwark
52960
Southwark
52961
Southwark
52962
Southwark
52963
Southwark
52964
Southwark
52965
Southwark
52966
Southwark
52967
Southwark
52968
Southwark
52969
Southwark
52970
Southwark
52971
Southwark
52972
Southwark
52973
Southwark
52974
Southwark
52975
Southwark
52976
Southwark
52977
Southwark
52978
Southwark
52979
Southwark
52980
Southwark
52981
Southwark
52982
Southwark
52983
Southwark
52984
Southwark
52985
Southwark
52986
Southwark
52987
Southwark
52988
Southwark
52989
Southwark
52990
Southwark
52991
Southwark
52992
Southwark
52993
Southwark
52994
Southwark
52995
Southwark
52996
Southwark
52997
Southwark
52998
Southwark
52999
Southwark
53000
Southwark
53001
Southwark
53002
Southwark
53003
Southwark
53004
Southwark
53005
Southwark
53006
Southwark
53007
Southwar

Southwark
53459
Southwark
53460
Southwark
53461
Southwark
53462
Southwark
53463
Southwark
53464
Southwark
53465
Southwark
53466
Southwark
53467
Southwark
53468
Southwark
53469
Southwark
53470
Southwark
53471
Southwark
53472
Southwark
53473
Southwark
53474
Southwark
53475
Southwark
53476
Southwark
53477
Southwark
53478
Southwark
53479
Southwark
53480
Southwark
53481
Southwark
53482
Southwark
53483
Southwark
53484
Southwark
53485
Southwark
53486
Southwark
53487
Southwark
53488
Southwark
53489
Southwark
53490
Southwark
53491
Southwark
53492
Southwark
53493
Southwark
53494
Southwark
53495
Southwark
53496
Southwark
53497
Southwark
53498
Southwark
53499
Southwark
53500
Southwark
53501
Southwark
53502
Southwark
53503
Southwark
53504
Southwark
53505
Southwark
53506
Southwark
53507
Southwark
53508
Southwark
53509
Southwark
53510
Southwark
53511
Southwark
53512
Southwark
53513
Southwark
53514
Southwark
53515
Southwark
53516
Southwark
53517
Southwark
53518
Southwark
53519
Southwark
53520
Southwar

Southwark
53978
Southwark
53979
Southwark
53980
Southwark
53981
Southwark
53982
Southwark
53983
Southwark
53984
Southwark
53985
Southwark
53986
Southwark
53987
Southwark
53988
Southwark
53989
Southwark
53990
Southwark
53991
Southwark
53992
Southwark
53993
Southwark
53994
Southwark
53995
Southwark
53996
Southwark
53997
Southwark
53998
Southwark
53999
Southwark
54000
Southwark
54001
Southwark
54002
Southwark
54003
Southwark
54004
Southwark
54005
Southwark
54006
Southwark
54007
Southwark
54008
Southwark
54009
Southwark
54010
Southwark
54011
Southwark
54012
Southwark
54013
Southwark
54014
Southwark
54015
Southwark
54016
Southwark
54017
Southwark
54018
Southwark
54019
Southwark
54020
Southwark
54021
Southwark
54022
Southwark
54023
Southwark
54024
Southwark
54025
Southwark
54026
54027
Southwark
54028
54029
Southwark
54030
Southwark
54031
Southwark
54032
Southwark
54033
Southwark
54034
Southwark
54035
Southwark
54036
Southwark
54037
Southwark
54038
Lambeth
54039
Southwark
54040
Southwark
5404

Southwark
54492
Southwark
54493
Southwark
54494
Southwark
54495
Southwark
54496
Southwark
54497
Southwark
54498
Southwark
54499
Southwark
54500
Southwark
54501
Southwark
54502
Southwark
54503
Southwark
54504
Southwark
54505
Southwark
54506
Southwark
54507
Southwark
54508
Southwark
54509
Southwark
54510
Southwark
54511
Southwark
54512
Southwark
54513
Southwark
54514
Southwark
54515
Southwark
54516
Southwark
54517
Southwark
54518
Southwark
54519
Southwark
54520
Southwark
54521
Southwark
54522
Southwark
54523
Southwark
54524
Southwark
54525
Southwark
54526
Southwark
54527
Southwark
54528
Southwark
54529
Southwark
54530
Southwark
54531
Southwark
54532
Southwark
54533
Southwark
54534
Southwark
54535
Southwark
54536
Southwark
54537
Southwark
54538
Southwark
54539
Southwark
54540
Southwark
54541
Southwark
54542
Southwark
54543
Southwark
54544
Southwark
54545
Southwark
54546
Southwark
54547
Southwark
54548
Southwark
54549
Southwark
54550
Southwark
54551
Southwark
54552
Southwark
54553
Southwar

Sutton
55030
Sutton
55031
Sutton
55032
Sutton
55033
Sutton
55034
Sutton
55035
Merton
55036
Merton
55037
Sutton
55038
Merton
55039
Sutton
55040
Sutton
55041
Sutton
55042
Sutton
55043
Sutton
55044
Sutton
55045
Sutton
55046
Sutton
55047
Sutton
55048
Sutton
55049
Sutton
55050
Sutton
55051
Sutton
55052
Sutton
55053
Sutton
55054
Sutton
55055
Sutton
55056
Sutton
55057
Sutton
55058
Sutton
55059
Sutton
55060
Sutton
55061
Sutton
55062
Sutton
55063
Sutton
55064
Sutton
55065
Sutton
55066
Sutton
55067
Sutton
55068
Sutton
55069
Sutton
55070
Sutton
55071
Sutton
55072
Sutton
55073
Sutton
55074
Sutton
55075
Sutton
55076
Sutton
55077
Sutton
55078
Sutton
55079
Sutton
55080
Sutton
55081
Sutton
55082
Sutton
55083
Sutton
55084
Sutton
55085
Sutton
55086
Sutton
55087
Sutton
55088
Sutton
55089
Sutton
55090
Sutton
55091
Sutton
55092
Sutton
55093
Sutton
55094
Sutton
55095
Sutton
55096
Sutton
55097
Sutton
55098
Sutton
55099
Sutton
55100
Sutton
55101
Sutton
55102
Sutton
55103
Sutton
55104
Sutton
55105
Sutton
55106

In [ ]:
dfs[1]["Borough"] = dfs[1].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[1].to_csv('clean_met_2021_02.csv', index=False)

In [ ]:
dfs[2]["Borough"] = dfs[2].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[2].to_csv('clean_met_2021_03.csv', index=False)

In [ ]:
dfs[3]["Borough"] = dfs[3].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[3].to_csv('clean_met_2021_04.csv', index=False)

In [ ]:
dfs[4]["Borough"] = dfs[4].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[4].to_csv('clean_met_2021_05.csv', index=False)

In [ ]:
dfs[5]["Borough"] = dfs[5].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[5].to_csv('clean_met_2021_06.csv', index=False)

In [ ]:
dfs[6]["Borough"] = dfs[6].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[6].to_csv('clean_met_2021_07.csv', index=False)

In [ ]:
dfs[7]["Borough"] = dfs[7].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[7].to_csv('clean_met_2021_08.csv', index=False)

In [ ]:
dfs[8]["Borough"] = dfs[8].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[8].to_csv('clean_met_2021_09.csv', index=False)

In [ ]:
dfs[9]["Borough"] = dfs[9].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[9].to_csv('clean_met_2021_10.csv', index=False)

In [ ]:
dfs[10]["Borough"] = dfs[10].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[10].to_csv('clean_met_2021_11.csv', index=False)

In [ ]:
dfs[11]["Borough"] = dfs[11].apply(get_location, axis=1)

# export the dataframe to a csv file
dfs[11].to_csv('clean_met_2021_12.csv', index=False)